## Deploy the winning model - RF

In [21]:
import pickle                                           # save the models to a file
with open (r"Final RF.pickle", "rb") as input_file:     # load RF from file to memory.
        RF = pickle.load(input_file)

Load the winning model into memory 

In [6]:
import pandas as pd                                     # load pandas, numpy, seaborn and matplotlib for this project.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

deploy = pd.read_csv('validation.csv')                      # Read the data from the file.
deploy.head(2)                                           # see the headers of the data

,Week,Month,Day,Hour,Stoch_m15,Stoch_h1,Stoch_h4,BBand_m15,BBand_h1,BBand_h4,Want
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89,False
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89,False


Deploy data have problems with column names. remediating that...

In [7]:
deploy.columns = ['week','month','day','hour','stoch_m15','stoch_h1','stoch_h4','bband_m15','bband_h1','bband_h4','want']

In [8]:
deploy.head(2)                                           # see the headers of the data

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4,want
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89,False
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89,False


Deploy data have problems with column names. fixed!

In [9]:
# change the want column to 0's and 1's

#split the want to 'true' and 'false'. save series in want_n
want_n = pd.get_dummies(deploy.want)
#print (want_n)

# concat want_n to the dataset
deploy = pd.concat([deploy,want_n], axis =1 )
deploy.merge(want_n,left_index=True, right_index=True)
deploy.drop(columns=['want'], inplace = True)
deploy.drop(columns=[False], inplace = True)
deploy.rename(columns={True: 'want'}, inplace=True)
print(deploy.head(3))

y_true = deploy.want

count0=0
count1=0

for i in y_true:
    if i == 0:
        count0 += 1
    else:
        count1 += 1

print('\nThis is y_true. saving this information for future use.')
print("0: " + str(count0) + "\n" + "1:  " + str(count1),'\n')

   week  month  day  hour  stoch_m15  stoch_h1  stoch_h4  bband_m15  bband_h1  \
0    39      9    1     0      82.58     72.25      8.48      90.85     30.56   
1    39      9    1     0      56.00     72.25      8.48      18.97     30.56   
2    39      9    1     0      20.16     72.25      8.48      75.90     30.56   

   bband_h4  want  
0     37.89     0  
1     37.89     0  
2     37.89     0  

This is y_true. saving this information for future use.
0: 1301
1:  125 



In [10]:
deploy.drop(columns=['want'], inplace = True)           # drop the want column

In [11]:
deploy.head(2)                                           # see the headers of the data

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89


make predictions....

In [22]:
y_pred_RF = RF.predict(deploy)                          # predict the y
proba_RF =   RF.predict_proba(deploy)                   # compute the probabilies..
proba_RF = proba_RF[:,1]
print ('y_pred_RF shape :' , y_pred_RF.shape)           #resulting y_pred_RF is a 1D numpy arrays of 0s and 1s. 
print ('y_pred_RF contents : ', y_pred_RF)
print (' proba_RF contents : ', proba_RF)
print ('\n' )

y_pred_RF shape : (1426,)
y_pred_RF contents :  [0 0 0 ... 0 0 0]
 proba_RF contents :  [0.1 0.1 0.  ... 0.1 0.2 0.2]




In [23]:
count0=0
count1=0

for i in y_pred_RF :
    if i == 0:
        count0 += 1
    else:
        count1 += 1

print('This is the predicted y column. ')        
print("0: " + str(count0) + "\n" + "1: " + str(count1),'\n')

This is the predicted y column. 
0: 1397
1: 29 



In [24]:
from sklearn import metrics

print ('Random Forest Scores : ' )
print ('\nConfusion Matrix' )
print (metrics.confusion_matrix(y_true=y_true, y_pred=y_pred_RF) )      # confusion matrix
print ('\nClassification Report' )
print (metrics.classification_report(y_true, y_pred_RF))                # classification report for RF
print ('-------------------------------------------------------')
print ('\n' )

Random Forest Scores : 

Confusion Matrix
[[1278   23]
 [ 119    6]]

Classification Report
             precision    recall  f1-score   support

          0       0.91      0.98      0.95      1301
          1       0.21      0.05      0.08       125

avg / total       0.85      0.90      0.87      1426

-------------------------------------------------------




make predictions. done. 
as we can see __Precision__ is really bad. 0.21 % !

saving the prediciton in a file with the model features for future use. 

In [25]:
deploy.head(2)

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89


In [26]:
columns = ['want']
df = pd.DataFrame(y_pred_RF,columns=columns)

df.want.head(2)

0    0
1    0
Name: want, dtype: uint8

In [27]:
final_result_for_saving_to_file = pd.concat([deploy,df.want], axis =1 )           # concat want_n to the dataset

In [28]:
deploy.head(2)

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89


In [29]:
final_result_for_saving_to_file.head(2)

,week,month,day,hour,stoch_m15,stoch_h1,stoch_h4,bband_m15,bband_h1,bband_h4,want
0,39,9,1,0,82.58,72.25,8.48,90.85,30.56,37.89,0
1,39,9,1,0,56.00,72.25,8.48,18.97,30.56,37.89,0


In [30]:
#final_result_for_saving_to_file.to_csv('deploy.csv')